This is a starter notebook for the project, you'll have to import the libraries you'll need, you can find a list of the ones available in this workspace in the requirements.txt file in this workspace. 

## Step 1: Setting Up the Python Application

In [1]:
# Load all relevant libraries
import os
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
import chromadb
import openai

# Setting up the OpenAI API key
os.environ["OPENAI_API_KEY"] = "YOUR API KEY"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

## Step 2: Generating Real Estate Listings

In [2]:
#Select and define parameters for the LLM model 
model_name = "gpt-3.5-turbo"
temperature = 0.0
llm = OpenAI(model_name=model_name, temperature=temperature, max_tokens = 4000)

# Generate real-estate listings
data_gen_template="""
generate csv formatted reviews for ten different real estate properties located in Seattle, Washington. 
for each property, generate {num_reviews} reviews. 
each review will have these fields in the csv: Neighborhood, Price in USD, No. of Bedrooms, No. of Bathrooms, House size in square feet, description limited to 500 characters and neighbourhood description limited to 500 characters
The descriptions need to be creative, csv format is a must.
"""
data_gen_prompt = PromptTemplate.from_template(data_gen_template)

#Print generated listings
print(llm(data_gen_prompt.format(num_reviews = 10)))

/opt/conda/lib/python3.10/site-packages/langchain/llms/openai.py:202: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/langchain/llms/openai.py:790: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


Neighborhood,Price in USD,No. of Bedrooms,No. of Bathrooms,House size in square feet,Description,Neighborhood Description
Capitol Hill,500000,2,1,1000,"This charming bungalow in the heart of Capitol Hill is the perfect starter home. With two cozy bedrooms, a spacious living room, and a beautifully landscaped backyard, this property is ideal for young professionals or a small family. Don't miss out on this gem!",Capitol Hill is known for its vibrant arts scene, trendy boutiques, and diverse dining options. Residents enjoy easy access to public transportation and stunning views of the city skyline.
Ballard,700000,3,2,1500,"This modern townhouse in Ballard offers the perfect blend of style and comfort. With three bedrooms, two bathrooms, and a sleek open-concept living space, this property is perfect for those who love to entertain. The rooftop deck is the perfect spot to enjoy a glass of wine while taking in the views of the Puget Sound.",Ballard is a hip neighborhood known for its craft

In [3]:
# Generate real estate listings
csv_content = llm(data_gen_prompt.format(num_reviews=10))

# Save the generated CSV content to a file
with open("real_estate_listings.csv", "w") as file:
    file.write(csv_content)

print("CSV file saved as 'real_estate_listings.csv'")


CSV file saved as 'real_estate_listings.csv'


In [4]:
# Parse CSV listings 
def parse_csv(file_path):
    with open(file_path, "r") as file:
        lines = file.readlines()
    
    headers = lines[0].strip().split(",")
    listings = []
    
    for line in lines[1:]:
        values = line.strip().split(",")
        listing = dict(zip(headers, values))
        listings.append(listing)
    
    return listings

listings = parse_csv("real_estate_listings.csv")

## Step 3: Storing Listings in a Vector Database

In [5]:
# Function to generate embeddings using OpenAI API
def get_embedding(text):
    response = openai.Embedding.create(
        input=text,
        model="text-embedding-ada-002"
    )
    return response['data'][0]['embedding']

# Initialize ChromaDB
db = chromadb.Client()
collection = db.get_or_create_collection(name="real_estate_listings")

# Store each listing in the vector database
for idx, listing in enumerate(listings):
    listing_text = f"Neighborhood: {listing['Neighborhood']}, Price: {listing['Price in USD']}, Bedrooms: {listing['No. of Bedrooms']}, Bathrooms: {listing['No. of Bathrooms']}, House Size: {listing['House size in square feet']}, Description: {listing['Description']}, Neighborhood Description: {listing['Neighborhood Description']}"
    embedding = get_embedding(listing_text)
    collection.add(
        documents=[listing_text],
        embeddings=[embedding],
        ids=[str(idx)]  # Add a unique ID for each document
    )

print("Listings stored in the vector database.")

Listings stored in the vector database.


## Step 4: Building the User Preference Interface

In [6]:
# Collect buyer preferences
questions = [   
    "How big do you want your house to be?", 
    "What are the 3 most important things for you in choosing this property?", 
    "Which amenities would you like?", 
    "Which transportation options are important to you?",
    "How urban do you want your neighborhood to be?",   
]

answers = [
    "A comfortable three-bedroom house with a spacious kitchen and a cozy living room.",
    "A quiet neighborhood, good local schools, and convenient shopping options.",
    "A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.",
    "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
    "A balance between suburban tranquility and access to urban amenities like restaurants and theaters."
]

# Parse and structure preferences
preferences = " ".join(answers)

# Generate embedding for buyer preferences
preference_embedding = get_embedding(preferences)

## Step 5: Searching Based on Preferences 

In [7]:
# Perform semantic search in the vector database
results = collection.query(
    query_embeddings=[preference_embedding],
    n_results=5  # Retrieve top 5 listings
)

# Display search results
for idx, result in enumerate(results['documents'][0]):
    print(f"Result {idx+1}:\n{result}\n")

Result 1:
Neighborhood: Ballard, Price: 700000, Bedrooms: 3, Bathrooms: 2, House Size: 1500, Description: "This modern townhouse in Ballard offers sleek design and ample space for a growing family. With three bedrooms, Neighborhood Description:  two bathrooms

Result 2:
Neighborhood: Greenwood, Price: 550000, Bedrooms: 3, Bathrooms: 2, House Size: 1400, Description: "Welcome home to this charming craftsman in Greenwood. With three bedrooms, Neighborhood Description:  two bathrooms

Result 3:
Neighborhood: University District, Price: 650000, Bedrooms: 3, Bathrooms: 2, House Size: 1300, Description: "This charming townhouse in the University District is a rare find. With three bedrooms, Neighborhood Description:  two bathrooms

Result 4:
Neighborhood: Fremont, Price: 600000, Bedrooms: 2, Bathrooms: 1, House Size: 1200, Description: "This cozy cottage in Fremont is a hidden gem waiting to be discovered. Featuring two bedrooms, Neighborhood Description:  one bathroom

Result 5:
Neighborhoo

## Step 6: Personalizing Listing Descriptions 

In [8]:
# Function to personalize listing
def personalize_listing(listing, preferences):
    prompt = f"""
    Here is a real estate listing:
    {listing}
    
    Based on the following buyer preferences:
    {preferences}
    
    Personalize the listing description to resonate with the buyer’s specific preferences without altering factual information.
    """
    response = llm(prompt)
    return response

# Personalize each retrieved listing
personalized_listings = []
for idx, result in enumerate(results['documents'][0]):
    personalized_listing = personalize_listing(result, preferences)
    personalized_listings.append(personalized_listing)
    print(f"Personalized Listing {idx+1}:\n{personalized_listing}\n")

Personalized Listing 1:
Welcome to your dream home in the heart of Ballard! This modern townhouse offers three spacious bedrooms, perfect for your growing family. The sleek design includes a spacious kitchen, ideal for cooking family meals, and a cozy living room for relaxing evenings together.

Located in a quiet neighborhood with good local schools and convenient shopping options, this home also features a backyard for gardening and a two-car garage for all your storage needs. Stay comfortable year-round with the modern, energy-efficient heating system.

Enjoy easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads for your daily commute. And with a perfect balance between suburban tranquility and access to urban amenities like restaurants and theaters, you'll have everything you need right at your fingertips. Don't miss out on this perfect blend of comfort and convenience in Ballard!

Personalized Listing 2:
Welcome home to this charming craftsman i